In [1]:
# Import modules and subroutines:

include("../subroutines/Subroutines.jl");

3.10.9


In [2]:
# Load the chemical data into julia as an array of chemical data structs:
# (one struct obtained for each molecular geometry as set up in the config file)

bond_lengths = [1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0]

mol_list = ["1p8","2p0","2p2","2p4","2p6","2p8","3p0"]

md_list = [
    "h2o_1p8A_sto-3g_021423%124203.hdf5",
    "h2o_2p0A_sto-3g_021423%124203.hdf5",
    "h2o_2p2A_sto-3g_021423%124204.hdf5",
    "h2o_2p4A_sto-3g_021423%124205.hdf5",
    "h2o_2p6A_sto-3g_021423%124206.hdf5",
    "h2o_2p8A_sto-3g_021423%124207.hdf5",
    "h2o_3p0A_sto-3g_021423%124208.hdf5"
]

chemical_data_list = []

for (m,mol) in enumerate(md_list)

    cdata_list = ReadIn("../datasets/pyscf_data/"*mol);
    push!(chemical_data_list, cdata_list[1])
    PrintChemData(chemical_data_list[m])
    
end

Molecule name: h2o_1p8A
Basis set: sto-3g
Molecular geometry: ../configs/xyz_files/h2o_double_dissoc/1p8.xyz
RHF energy: -74.51129616870288
FCI energy: -74.79063381243292
Molecule name: h2o_2p0A
Basis set: sto-3g
Molecular geometry: ../configs/xyz_files/h2o_double_dissoc/2p0.xyz
RHF energy: -74.4012364583995
FCI energy: -74.76201009070273
Molecule name: h2o_2p2A
Basis set: sto-3g
Molecular geometry: ../configs/xyz_files/h2o_double_dissoc/2p2.xyz
RHF energy: -74.3304107372596
FCI energy: -74.74829392590973
Molecule name: h2o_2p4A
Basis set: sto-3g
Molecular geometry: ../configs/xyz_files/h2o_double_dissoc/2p4.xyz
RHF energy: -74.2963321563054
FCI energy: -74.74206586837659
Molecule name: h2o_2p6A
Basis set: sto-3g
Molecular geometry: ../configs/xyz_files/h2o_double_dissoc/2p6.xyz
RHF energy: -74.2801471741282
FCI energy: -74.73939424668808
Molecule name: h2o_2p8A
Basis set: sto-3g
Molecular geometry: ../configs/xyz_files/h2o_double_dissoc/2p8.xyz
RHF energy: -74.27080055674686
FCI energ

In [3]:
# Standardized sweep objects:
opt_sweeps30 = Sweeps(30)
maxdim!(opt_sweeps30,30)
mindim!(opt_sweeps30,30)
cutoff!(opt_sweeps30,1e-12)
setnoise!(opt_sweeps30, 1e-3,1e-4,1e-5,1e-6, 1e-7)

opt_sweeps8 = Sweeps(30)
maxdim!(opt_sweeps8,8)
mindim!(opt_sweeps8,8)
cutoff!(opt_sweeps8,1e-12)
setnoise!(opt_sweeps8, 1e-3,1e-4,1e-5,1e-6, 1e-7)

opt_sweeps3 = Sweeps(30)
maxdim!(opt_sweeps3,3)
mindim!(opt_sweeps3,3)
cutoff!(opt_sweeps3,1e-12)
setnoise!(opt_sweeps3, 1e-3,1e-4,1e-5,1e-6, 1e-7)

# Geometry optimization parameters:
gp0 = GeomParameters(
    a_maxiter=300000,
    eta=-2,
    shrp=2.0,
    a_alpha=1e1
)

# Number of MultiGeomOptim repetitions:
nreps = 4

op1 = OptimParameters(
    maxiter=1, 
    numloop=1, 
    numopt=5,
    noise=[0.0], 
    sd_thresh="inversion", 
    sd_eps=1.0e-8,
    delta=[0.0],
    afunc="flat",
    ttol=0.25,
    theta=0.0
)

op2 = OptimParameters(
    maxiter=2, 
    numloop=1, 
    numopt=-1,
    noise=[0.0], 
    sd_thresh="inversion", 
    sd_eps=1.0e-8,
    delta=[0.0],
    ttol=0.2,
    theta=0.0
)

op3 = OptimParameters(
    maxiter=1, 
    numloop=1, 
    numopt=-1,
    noise=[0.0], 
    sd_thresh="inversion", 
    sd_eps=1.0e-8,
    delta=[0.0],
    theta=0.0
);

In [ ]:
# Collect the various estimates:

e_dmrg8_vec = []
e_dmrg3_vec = []
e_nomps_vec = []
e_tnqe_vec = []
Ipq_vec = []

# Repeat for each geometry:
for m=1:length(mol_list)
    
    cdata = chemical_data_list[m]
    
    println("\n\n##### MOLECULAR GEOMETRY $(m)/$(length(mol_list)): #####\n\n")
    
    # This DMRG object is for determining bipartition entropies:
    dmrg1 = GenSubspace(cdata, 1, psi_maxdim=30);
    dmrg1.ord_list = [collect(1:dmrg1.chem_data.N_spt) for j=1:dmrg1.mparams.M]
    
    GenStates!(dmrg1, verbose=true, sweeps=opt_sweeps30);
    GenPermOps!(dmrg1, verbose=true)
    GenSubspaceMats!(dmrg1, verbose=true)
    
    S1, S2, Ipq = MutualInformation(dmrg1.psi_list[1], dmrg1.chem_data)
    
    push!(Ipq_vec, Ipq)
    
    opt_ord = InfDistAnnealing(
        Ipq, 
        1, 
        gp0,
        verbose=false
    )[1]
    
    println("\n\n##### DMRG m=8 #####\n")
    # DMRG with bond dimension 8:
    dmrg8 = GenSubspace(cdata, 1, psi_maxdim=8)
    dmrg8.ord_list = [opt_ord]
    
    GenPermOps!(dmrg8, verbose=true)
    GenStates!(dmrg8, sweeps=opt_sweeps8, verbose=true)
    GenSubspaceMats!(dmrg8, verbose=true)
    
    push!(e_dmrg8_vec, dmrg8.H_mat[1,1]+cdata.e_nuc)
    println(e_dmrg8_vec[end])
    
    
    println("\n\n##### DMRG m=3 #####\n")
    # DMRG with bond dimension 3:
    dmrg3 = GenSubspace(cdata, 1, psi_maxdim=3)
    dmrg3.ord_list = [opt_ord]
    
    GenPermOps!(dmrg3, verbose=true)
    GenStates!(dmrg3, sweeps=opt_sweeps3, verbose=true)
    GenSubspaceMats!(dmrg3, verbose=true)
    
    push!(e_dmrg3_vec, dmrg3.H_mat[1,1]+cdata.e_nuc)
    println(e_dmrg3_vec[end])
    
    
    println("\n\n##### NOMPS m=3 #####\n")
    # NOMPS with 5 states, bond dimension 3:
    nomps = GenSubspace(cdata, 5, psi_maxdim=3)
    nomps.ord_list = [opt_ord for j=1:nomps.mparams.M]
    
    GenPermOps!(nomps, verbose=true)
    GenHams!(nomps)
    GenStates!(nomps, randomize=true, verbose=true)
    GenSubspaceMats!(nomps, verbose=true)
    SolveGenEig!(nomps, thresh="inversion", eps=1e-8, verbose=false)
    
    """
    # Optimize:
    TwoSitePairSweep!(
        nomps,
        op1,
        jpairs=[[1,2],[2,3],[3,4],[4,5],[1,5]],
        verbose=true
    )
    """
    
    MultiGeomOptim!(
        nomps,
        [op1, op2, op3],
        reps = nreps,
        rep_struct = [
            ["twositepair", 1],
            ["twosite", 2]
        ],
        verbose=true
    )
    
    OneSitePairSweep!(
        nomps,
        op3,
        verbose=true
    )
    
    push!(e_nomps_vec, nomps.E[1]+cdata.e_nuc)
    println(e_nomps_vec[end])
    
    println("\n\n##### TNQE m=3 #####\n")
    # TNQE with 5 states, bond dimension 3:
    tnqe1 = GenSubspace(cdata, 5, psi_maxdim=3)
    
    
    tnqe1.ord_list = InfDistAnnealing(
        Ipq, 
        tnqe1.mparams.M, 
        gp0,
        verbose=false
    )
    
    GenPermOps!(tnqe1, verbose=true)
    GenHams!(tnqe1)
    GenStates!(tnqe1, randomize=true, verbose=true)
    
    GenSubspaceMats!(tnqe1, verbose=true)
    SolveGenEig!(tnqe1, thresh="inversion", eps=1e-8, verbose=false)
    
    """
    # Optimize:
    TwoSitePairSweep!(
        tnqe1,
        op1,
        jpairs=[[1,2],[2,3],[3,4],[4,5],[1,5]],
        verbose=true
    )
    """
    
    MultiGeomOptim!(
        tnqe1,
        [op1, op2, op3],
        reps = nreps,
        rep_struct = [
            ["twositepair", 1],
            ["twosite", 2]
        ],
        verbose=true
    )
    
    OneSitePairSweep!(
        tnqe1,
        op3,
        verbose=true
    )
    
    push!(e_tnqe_vec, tnqe1.E[1]+cdata.e_nuc)
    println(e_tnqe_vec[end])
    
end



##### MOLECULAR GEOMETRY 1/7: #####


Generating states:
Progress: [1/1] 
Done!
Generating permutation operators:

Done!



##### DMRG m=8 #####

Generating permutation operators:

Done!

Generating states:
Progress: [1/1] 
Done!
-74.79038891254035


##### DMRG m=3 #####

Generating permutation operators:

Done!

Generating states:
Progress: [1/1] 
Done!
-74.67519814847857


##### NOMPS m=3 #####

Generating permutation operators:
Progress: [10/10] 
Done!

Generating states:
Progress: [5/5] 
Done!

----| Starting MultiGeomOptim!


----| Repetition 1/4:


GENERALIZED TWO-SITE SWEEP ALGORITHM:
Iter: 1/1; pair: 5/5; sweep: 1/1; bond: 6/6; E_min = -79.61644; kappa = 130.0     
Done!


GENERALIZED TWO-SITE SWEEP ALGORITHM:
Iter: 2/2; state: 5/5; sweep: 1/1; bond: 6/6; E_min = -79.61944; kappa = 13.8      
Done!


----| Repetition 1/4 complete!
----| E_new = -79.61944325945556; E_best = -79.61944325945556
----| Outcome = "accept!"; acceptance ratio = 100%


----| Repetition 2/4:


GENERALI

In [ ]:
e_rhf_vec = [chemical_data_list[m].e_rhf for m=1:length(chemical_data_list)]
e_fci_vec = [chemical_data_list[m].e_fci for m=1:length(chemical_data_list)]

plot(
    bond_lengths, 
    e_rhf_vec, 
    color=RGB(0.7,0.7,0.7), 
    linewidth=2.0,
    label="RHF",
    title="Double-dissociation of H2O in STO-3G basis (M=5,m=3)",
    titlefontsize=11.0
)

plot!(
    bond_lengths, 
    e_fci_vec, 
    color=RGB(0.3,0.3,0.3), 
    linewidth=2.0,
    label="FCI"
)


plot!(
    bond_lengths, 
    e_dmrg8_vec, 
    color=RGB(0.4,0.4,1.0), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="DMRG; m=8"
)

plot!(
    bond_lengths, 
    e_dmrg3_vec, 
    color=RGB(0.3,0.8,0.5), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="DMRG; m=3"
)

plot!(
    bond_lengths, 
    e_nomps_vec, 
    color=color=RGB(1.0,0.8,0.3), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="Cl. NOMPS"
)

plot!(
    bond_lengths, 
    e_tnqe_vec, 
    color=color=RGB(1.0,0.4,0.4), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="TNQE (NOMPS)"
)


In [ ]:
plot(
    bond_lengths, 
    e_fci_vec, 
    color=RGB(0.3,0.3,0.3), 
    linewidth=2.0,
    label="FCI"
)

plot!(
    bond_lengths, 
    e_nomps_vec, 
    color=color=RGB(1.0,0.8,0.3), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="Cl. NOMPS"
)

plot!(
    bond_lengths, 
    e_tnqe_vec, 
    color=color=RGB(1.0,0.4,0.4), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="TNQE (NOMPS)"
)

plot!(
    bond_lengths, 
    e_dmrg8_vec, 
    color=RGB(0.4,0.4,1.0), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="DMRG; m=8"
)

plot!(
    bond_lengths, 
    e_dmrg3_vec, 
    color=RGB(0.3,0.8,0.5), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="DMRG; m=3"
)

In [ ]:
plot(
    bond_lengths,
    e_fci_vec.+0.0016,
    fillrange = e_fci_vec.-0.0016,
    color = RGB(0.3,0.8,0.5),
    fillalpha = 0.2,
    label = "Chem. acc.",
    title="Double-dissociation of H2O in STO-3G basis (M=5,m=3)",
    titlefontsize=11.0
)

plot!(
    bond_lengths,
    e_fci_vec.-0.0016,
    color = RGB(0.3,0.8,0.5),
    primary=false
)

plot!(
    bond_lengths, 
    e_fci_vec, 
    color=RGB(0.3,0.3,0.3), 
    linewidth=2.0,
    label="FCI"
)

plot!(
    bond_lengths, 
    e_nomps_vec, 
    color=color=RGB(1.0,0.8,0.3), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="Cl. NOMPS"
)

plot!(
    bond_lengths, 
    e_tnqe_vec, 
    color=color=RGB(1.0,0.4,0.4), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="TNQE (NOMPS)"
)


In [ ]:
"""
plot!(
    bond_lengths, 
    e_dmrg8_vec, 
    color=RGB(0.4,0.4,1.0), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="DMRG; m=8"
)
"""

In [ ]:
for Ipq in Ipq_vec
    
    OrbitalGraphPlot(Ipq, multiplier=3e1)
    
end